# Pinecone - Creando un asistente conversacional

### Arquitectura RAG

1. [Introducción](#intro)
2. [LangChain](#langchain)
3. [Embeddings](#embeddings)
4. [Carga de los Embeddings en Pinecone](#persist)
5. [Integración con OpenAI y generación de prompts](#openai)

## Introducción <a name="intro"></a>

El propósito general de este notebook es generar un asistente conversacional basado en la arquitectura RAG (_Retrieval Augmented Generation_), es decir, nosotros dispondremos de unos documentos PDFs que serán nuestra base de conocimiento (_knowledgebase_) almacenados como Embeddings dentro de Pinecone, posteriormente, a través de LangChain conectaremos los modelos GPT para dotar al sistema de la capacidad de lanzar querys en forma de procesamiento del lenguaje natural que este, se conecte con la Embedding del usuario a la BD Vectorial y, sea capaz de devolver información detallada sobre nuestros propios documentos mediante IA Generativa con los modelos GPT

## LangChain <a name="langchain"></a>

LangChain (https://python.langchain.com/docs/get_started/introduction) se trata de una iniciativa Open Source que nos permite aobrdar uno de los problemas más recurrentes cuando trabajamos con LLMs que no es ni más ni menos que "poder entablar una conversación".

Como tal, podemos hacer peticiones a un modelo GPT una sola vez, este, sin problemas nos proveerá de una respuesta, pero, ¿Cómo podemos dotar a un sistema de la capacidad de enlazar (encadenar) una conversación? ahí es donde surge LangChain.

__LangChain__ es un marco para desarrollar aplicaciones basadas en modelos del lenguaje (únicamente se especializa en NLP), gracias a LangChain se consigue que:
+ Se creen modelos que sean _conscientes_ del contexto de una conversación.
+ Otorgar a un modelo Large Language Model la capacidad de razonar.

Para instalar LangChain en Python haremos:

```python
!pip install langchain
```

Además de encadenar conversaciones LangChain tiene una funcionalidad muy interesante que es la de leer archivos para su posterior procesamiento, no solo PDFs e incluso funciones para aplicar Transformers a dichos documentos. https://python.langchain.com/docs/modules/data_connection/

La función `document_loaders` https://python.langchain.com/docs/modules/data_connection/document_loaders/ tiene la capacidad de cargar los siguientes tipos de archivos:
+ CSV
+ Directorios
+ PDF
+ Markdown y texto
+ HTML
+ JSON

Importamos un archivo PDF, previamente necesitamos instalar una dependencia
```python
!pip install pypdf
```


In [1]:
from langchain.document_loaders import PyPDFLoader

# Iniciamos el objeto lector de PDF
loader = PyPDFLoader("An_Introduction_to_Logistic_Regression_From_Basic_.pdf")

# Lo cargamos
file   = loader.load()

# Vemos que contiene nuestro archivo
file[: 10]

[Document(page_content='© 2013 Korean Society of Nursing Science  www.kan.or.kr  |  ISSN 2005-3673INTRODUCTION\nMultivariable methods of statistical analysis commonly appear in \ngeneral health science literature (Bagley, White, & Golomb, 2001). The \nterms “multivariate analysis” and “multivariable analysis” are often used \ninterchangeably in the literature. In the strict sense, multivariate analysis \nrefers to simultaneously predicting multiple outcomes and multivariable \nanalysis uses multiple variables to predict a single outcome (Katz, 1999). \nThe multivariable methods explore a relation between two or more \npredictor (independent) variables and one outcome (dependent) vari-\nable. The model describing the relationship expresses the predicted value \nof the outcome variable as a sum of products, each product formed by \nmultiplying the value and coefficient of the independent variable. The \ncoefficients are obtained as the best mathematical fit for the specified \nmodel. A c

In [2]:
print("Total de elementos cargados --> ", len(file))

Total de elementos cargados -->  11


Vemos que se ha cargado un documento con información no estructurada, con tantas páginas como diapositivas, no obstante, es recomendable no pasar toda esta información de golpe a los modelos GPT, por lo que se recurre comúmente a técnicas de _chunking_ es decir, obtener pequeños fragmentos o porciones del documento (_chunks_) sobre los cuáles podamos ir trabajando en pequeños batches.

Para ir dividiendo la información del archivo PDF en pequeños fragmentos volvemos a emplear funciones de LangChain, en este caso, la función que podemos encontrar desde `text_splitter` https://python.langchain.com/docs/modules/data_connection/document_transformers/ `RecursiveCharacterTextSplitter()`

En otras palabras, con `RecursiveCharacterTextSplitter` lo que hace es, desde nuestro PDF ir dividiendo en párrafos, frases y palabras.

https://api.python.langchain.com/en/latest/text_splitter/langchain.text_splitter.RecursiveCharacterTextSplitter.html

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size      = 200,
    chunk_overlap   = 20,
    length_function = len,
    add_start_index = True,
)

In [4]:
# Ahora dividimos los documentos de nuestro PDF
chunks = text_splitter.split_documents(file)

In [5]:
print("Número de Chunks producidos desde nuestro PDF --> ", len(chunks))

Número de Chunks producidos desde nuestro PDF -->  320


## Embeddings <a name="embeddings"></a>

Una vez que tenemos separado en pequeños fragmentos nuestro documento PDF, ya podemos obtener los vectores de Word Embeddings sobre el mismo, para posteriormente, poder almacenarlos en Pinecone. 

Si realizamos todo este proceso de cero, tendríamos que pasar por una interesante tarea de limpieza de texto (recordar que, en cierto modo seguimos trabajando con datos raw) y, posteriormente entrenar un modelo de vectorización propio con Word2Vec, pero, de nuevo aparece LangChain para proporcionarnos una enorme gama de modelos pre-entrenados que son capaces de crear Embeddings, esto, se consigue desde las funciones `embeddings` https://python.langchain.com/docs/modules/data_connection/text_embedding/ desde LangChain podemos acceder a los modelos de Embeddings como:
+ HuggingFace
+ OpenAI
+ Bedrock (AWS)
+ PalM (Google)
+ spaCy
+ Ollama
+ Etc... https://python.langchain.com/docs/integrations/text_embedding/

Dada la dimensionalidad de nuestros vectores, vamos a cargar los Embeddings desde __HuggingFace__ https://python.langchain.com/docs/integrations/text_embedding/huggingfacehub

IMPORTANTE: Para buscar el modelo adecuado debemos investigar dentro de HuggingFace modelos que soporten creación de Embeddings, una opción puede ser los modelos de la familia sentence-transformers https://huggingface.co/sentence-transformers

En nuestro caso, vamos a cargar un modelo ligero como el multilingual `sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2` https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2

Este modelo tiene una dimensionalidad de 384 que es justo lo que estamos utilizando dentro de nuestra demo.

Para poder trabajar con los modelos de sentence-transformers debemos instalar previamente el paquete:
```python
!pip install sentence_transformers
```

In [6]:
from langchain.embeddings import HuggingFaceEmbeddings

emb_model = HuggingFaceEmbeddings(model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

Una vez descargado el modelo, podemos realizar alguna prueba para comprobar cómo realiza los Embeddings, esto, se consigue desde las funciones de codificación (encoder) y decodificación (decoder)

In [7]:
from sentence_transformers import SentenceTransformer

# Cargamos el modelo 
model = SentenceTransformer(model_name_or_path = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# Obtenemos Embeddings
emb_query = model.encode("Explícame qué es una regresión lineal")

In [8]:
print("Dimensionalidad de los Embeddings --> ", len(emb_query))
print(emb_query[:5])

Dimensionalidad de los Embeddings -->  384
[-0.48585448 -0.0671955   0.00969446 -0.0858185   0.32764772]


## Carga de los Embeddings en Pinecone <a name="persist"></a> 

Ahora, ya sabemos cómo vamos a conseguir qué preguntando en lenguaje natural, nuestras querys se puedan codificar como Embeddings, pero, todavía no hemos insertado nuestros documentos como Embeddings en Pinecone.

Para insertar documentos en Pinecone vuelve a ayudarnos notiramente LangChain, ya que tiene integraciones directas con múltiples bases de datos vectoriales https://python.langchain.com/docs/modules/data_connection/vectorstores/

Desde las funciones de `vectorsotres` podemos buscar la función que inicie la conexión con nuestro proveedor de base de datos

In [9]:
from langchain.vectorstores import Pinecone

La función de Pinecone (como vector store) que se encarga de poder almacenar información como Embeddings de forma automática, en nuestro caso de uso es `from_documents` ya que tenemos los chunks creados previamente, además debemos pasarle el nombre de nuestro índice de Pinecone y, el modelo ya creado de Embeddings.

NOTA: La conexión (`pinecone.init()`) debe estar activa antes de hacer ese paso

In [10]:
PINECONE_API_KEY = ""
PINECONE_ENV     = "gcp-starter"
INDEX_NAME       = "demo1"

import pinecone

pinecone.init( 
    api_key      = PINECONE_API_KEY,
    environment  = PINECONE_ENV
)

In [11]:
# Subimos nuestros documentos
Pinecone.from_documents(
    documents  = chunks,     # Chunks de nuestros documentos
    embedding  = emb_model,  # Modelo ya cargado de embeddings
    index_name = INDEX_NAME  # Nombre del Index Pinecone
)

## Integración con OpenAI y generación de prompts <a name="openai"></a> 

Lo primero de todo, debemos instalar la librería de OpenAI
```python
!pip install openai
```

Tal y como hemos hablado de LangChain, es el punto de unión entre la BD Vectorial (Pinecone) y el LLM (OpenAI), por lo tanto, de nuevo con LangChain obtendremos funciones que nos ayuden a conectar estos dos mundos, para este propósito necesitamos:
+ Generación del chat - De OpenAI https://python.langchain.com/docs/modules/model_io/chat
+ Tipo de chat - En un sistema RAG generalmente lo implementamos mediante _question and answering_ Q&A

In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain

Los pasos siguientes serían:
+ Definir la base de datos y el modelo de Embeddings
+ Lanzar el chat con un modelo LLM

In [13]:
# Definimos la conexión de la BD y el modelo de Embeddings

bbdd_vectorial = Pinecone.from_existing_index(
    index_name = INDEX_NAME, # Pinecone Index
    embedding  = emb_model   # Modelo de Embeddings
)

In [14]:
# Inicializamos el chat con un LLM
# https://api.python.langchain.com/en/latest/chat_models/langchain.chat_models.openai.ChatOpenAI.html
OPENAI_API_KEY = ""

# Configuración del chat
llm_model = ChatOpenAI(
    openai_api_key = OPENAI_API_KEY,
    model_name     = "gpt-3.5-turbo",  # Nombre del modelo de OpenAI https://platform.openai.com/docs/models
    temperature    = 0.8               # Entre 0 y 1, la creatividad del modelo (Alucinaciones)
)

# Conexión con LangChain - Tipo Q&A
chat_chain = load_qa_chain(
    llm       = llm_model,
   chain_type = "stuff"
)

Una vez definida la configuracíon general del chat, ya podemos comenzar a hacer preguntas a nuestro "propio _ChatGPT_" !! Existen diversas formas de crear _prompts_ en nuestro caso, vamos a implementarlo de una forma simple en dos fases:
1. Obteniendo la similaridad de vectores con `similarity_search`
2. Lanzando el promt con la función `chat_chain.run()`

In [16]:
ask = "¿How can i fit logistic regression model?"

# Similaridad
similarity = bbdd_vectorial.similarity_search(query = ask, 
                                              k     = 4)

# Prompt
prompt = chat_chain.run(question = ask, input_documents = similarity)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

Nota: Si tenemos el crédito gratuito agotado (al cabo de 3 meses aprecerá este error)
    
`Error Code 429 - You exceeded your current quota, please check your plan and billing details.`

In [ ]:
prompt